# 1.1 Installing The Rust Toolchain

Installing The Rust Toolchain
- https://rustup.rs/

In [ ]:
! curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

Release Channels
- stable
- beta
- nightly

What Toolchains Do We Need?
- `rustup update`: update your toolchains
- `rustup toolchain list`: give you an overview of what is installed on your system

In [2]:
! rustup update

info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: checking for self-update

  stable-x86_64-unknown-linux-gnu unchanged - rustc 1.81.0 (eeb90cda1 2024-09-04)

info: cleaning up downloads & tmp directories


In [1]:
! rustup toolchain list

stable-x86_64-unknown-linux-gnu (default)


# 1.2 Project Setup

- `rustc`: the Rust compiler itself

In [3]:
! rustc --version

rustc 1.81.0 (eeb90cda1 2024-09-04)


- `cargo` : Rust’s build tool

In [4]:
! cargo --version

cargo 1.81.0 (2dbb1af80 2024-08-20)


use `cargo` to create the skeleton of the project we will be working on for the whole book:

In [1]:
! cargo new zero2prod

    Creating binary (application) `zero2prod` package
note: see more `Cargo.toml` keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html


In [2]:
! tree

.
├── 01_Getting_Started.ipynb
├── LICENSE
├── README.md
└── zero2prod
    ├── Cargo.toml
    └── src
        └── main.rs

2 directories, 5 files


- git: Github Action

> https://docs.github.com/en/authentication/connecting-to-github-with-ssh

In [ ]:
! git config --global user.email "renew2049@outlook.com"

In [ ]:
! git config --global user.name "nl"

host the project on GitHub

In [5]:
! git add .

In [8]:
! git commit -am "Project skeleton"

[main eb746cd] Project skeleton
 4 files changed, 347 insertions(+), 1 deletion(-)
 create mode 100644 01_Getting_Started.ipynb
 create mode 100644 zero2prod/Cargo.toml
 create mode 100644 zero2prod/src/main.rs


In [11]:
! git remote add origin git@github.com:ningliu2024/zero2prod.git

error: remote origin already exists.


In [12]:
! git push -u origin main

To github.com:ningliu2024/notes_Zero_to_Production_in_Rust.git
   d5233a4..eb746cd  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


**GitHub Actions** feature for CI pipelines

# 1.3 IDEs

- Rust-analyzer: VSCode
- IntelliJ Rust, RustRover

# 1.4 Inner Development Loop

- Make a change;
- Compile the application;
- Run tests;
- Run the application.

Rust does not help us here - compilation speed can become a pain point on big projects.

## Faster Linking

In [ ]:
# .cargo/config.toml

# On Windows
# ```
# cargo install -f cargo-binutils
# rustup component add llvm-tools-preview
# ```
[target.x86_64-pc-windows-msvc]
rustflags = ["-C", "link-arg=-fuse-ld=lld"]
[target.x86_64-pc-windows-gnu]
rustflags = ["-C", "link-arg=-fuse-ld=lld"]

# On Linux:
# - Ubuntu, `sudo apt-get install lld clang`
# - Arch, `sudo pacman -S lld clang`
[target.x86_64-unknown-linux-gnu]
rustflags = ["-C", "linker=clang", "-C", "link-arg=-fuse-ld=lld"]

# On MacOS, `brew install llvm` and follow steps in `brew info llvm`
[target.x86_64-apple-darwin]
rustflags = ["-C", "link-arg=-fuse-ld=lld"]
[target.aarch64-apple-darwin]
rustflags = ["-C", "link-arg=-fuse-ld=/opt/homebrew/opt/llvm/bin/ld64.lld"]

> zld: Mac Apple

> This might not truly be the case though - *mold* is the newest linker on the block and it looks even faster than lld! It feels a bitearly, so we will not be using it as our default linker, but consider checking it out.

## cargo-watch

reducing the perceived compilation time - i.e. thetime you spend looking at your terminal waiting for `cargo check` or `cargo run` to complete.

In [14]:
! cargo install cargo-watch

    Updating crates.io index
     Ignored package `cargo-watch v8.5.2` is already installed, use --force to override


*cargo-watch*: monitors your source code to trigger commands every time a file changes.

In [ ]:
! cd zero2prod && cargo watch -x check

will run `cargo check` after every code change.<br/>
This reduces the perceived compilation time:
- you are still in your IDE, re-reading the code change you just made;
- **cargo-watch**, in the meantime, has already kick-started the compilation process;
- once you switch to your terminal, the compiler is already halfway through!

**cargo-watch** supports command chaining as well:

In [ ]:
! cargo watch -x check -x test -x run

- It will start by running `cargo check.`
- If it succeeds, it launches `cargo test`.
- If tests pass, it launches the application with `cargo run`.

Our inner development loop, right there!

# 1.5 Continuous Integration

Continuous Integration
- reduces the chances of having to sort out messy merge conflicts due tolong-lived branches.
- tightens the feedback loop